In [1]:
import pandas
import operator
import numpy as np
import datetime
hp_test = {}
for hp1 in range(2,5):
    for hp2 in (400,500,600,700):
        for hp3 in (1,2):
            for hp4 in (0,-.05,-.1):
                data = pandas.read_csv('games2012.csv')
                data['total']=data['PTS']+data['PTS.1']
                ID = {}
                outcomes = []
                for idx, row in data.iterrows():
                    if data.loc[idx,'PTS']>data.loc[idx,'PTS.1']:
                        winner = data.loc[idx,'Visitor/Neutral']; loser = data.loc[idx,'Home/Neutral'];
                        pd = data.loc[idx,'PTS'] - data.loc[idx,'PTS.1']; hm_i = 0
                    else:
                        winner = data.loc[idx,'Home/Neutral']; loser = data.loc[idx,'Visitor/Neutral'];
                        pd = data.loc[idx,'PTS.1'] - data.loc[idx,'PTS'];hm_i = 1
                    game_date = datetime.datetime.strptime(data.loc[idx]["Date"],"%a, %b %d, %Y")
                    if winner in ID.keys():
                        w_elo = ID[winner][0]
                        if (ID[winner][1] - game_date).days == 1: w_date = 1 
                        else: w_date = 0
                        w_games = ID[winner][2]
                    else: 
                        w_elo = 1500
                        w_date = 0
                        w_games = 0
                    if loser in ID.keys():
                        l_elo = ID[loser][0]
                        if (ID[loser][1] - game_date).days == 1: l_date = 1
                        else: l_date = 0
                        l_games = ID[loser][2]
                    else: 
                        l_elo = 1500
                        l_date = 0
                        l_games = 0
                    if hm_i == 1: w_elo = w_elo+hp3 + hp4*w_date; l_elo = l_elo + hp4*l_date
                    else: l_elo = l_elo +hp3 + hp4*l_date; w_elo = w_elo + hp4*l_date;
                    if w_elo>l_elo: outcomes.append(1)
                    if l_elo>w_elo: outcomes.append(0)
                    w_new_elo = (w_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(1-(1/(1+10**((l_elo-w_elo)/400)))))
                    l_new_elo = (l_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(0-(1/(1+10**((w_elo-l_elo)/400)))))
                    ID[winner]=(w_new_elo,game_date,w_games + 1);
                    ID[loser]=(l_new_elo,game_date,l_games + 1)
                hp_test[(hp1,hp2,hp3,hp4)] = (sum(outcomes)/float(len(outcomes)))
print sorted(hp_test.items(), key=operator.itemgetter(1),reverse=True)

[((4, 700, 2, -0.05), 0.6353535353535353), ((4, 400, 2, -0.05), 0.6353535353535353), ((4, 700, 2, -0.1), 0.6353535353535353), ((4, 500, 2, -0.05), 0.6353535353535353), ((4, 600, 2, -0.1), 0.6353535353535353), ((4, 500, 2, -0.1), 0.6353535353535353), ((4, 600, 2, -0.05), 0.6353535353535353), ((4, 400, 2, -0.1), 0.6353535353535353), ((3, 500, 2, -0.05), 0.6333333333333333), ((3, 600, 2, -0.1), 0.6333333333333333), ((3, 500, 2, -0.1), 0.6333333333333333), ((3, 700, 2, -0.1), 0.6333333333333333), ((3, 400, 2, -0.1), 0.6333333333333333), ((3, 700, 2, -0.05), 0.6333333333333333), ((3, 400, 2, -0.05), 0.6333333333333333), ((3, 600, 2, -0.05), 0.6333333333333333), ((2, 700, 1, -0.05), 0.6323232323232323), ((2, 600, 2, -0.1), 0.6323232323232323), ((4, 700, 1, 0), 0.6323232323232323), ((2, 700, 1, -0.1), 0.6323232323232323), ((4, 400, 1, 0), 0.6323232323232323), ((4, 600, 1, 0), 0.6323232323232323), ((2, 500, 2, -0.1), 0.6323232323232323), ((2, 700, 2, -0.05), 0.6323232323232323), ((2, 400, 2, -

In [42]:
print sorted(ID.items(), key=operator.itemgetter(1))

[('New York Knicks', 1429.1020865981511), ('Minnesota Timberwolves', 1429.8264966520815), ('Philadelphia 76ers', 1449.6889059015343), ('Los Angeles Lakers', 1452.8458649351214), ('Orlando Magic', 1481.041649560636), ('Sacramento Kings', 1505.7195279637463), ('Denver Nuggets', 1522.4209952449073), ('Charlotte Hornets', 1524.7164374552988), ('Detroit Pistons', 1547.3065579000677), ('Miami Heat', 1555.1196338937884), ('Brooklyn Nets', 1562.2676929516283), ('Phoenix Suns', 1566.935709631726), ('Milwaukee Bucks', 1581.4526295910337), ('Indiana Pacers', 1583.7306680480892), ('Boston Celtics', 1587.1195250155222), ('Washington Wizards', 1591.3868122903748), ('Utah Jazz', 1599.454372328689), ('New Orleans Pelicans', 1602.1887937926476), ('Toronto Raptors', 1612.6188414194928), ('Dallas Mavericks', 1624.9392707368092), ('Portland Trail Blazers', 1627.7719616507015), ('Oklahoma City Thunder', 1629.8439504046573), ('Chicago Bulls', 1630.0660556442467), ('Memphis Grizzlies', 1644.3861379004986), (

In [73]:
import datetime
import re
print -.1*((datetime.datetime.strptime(data.loc[1]["Date"],"%a, %b %d, %Y"))-(datetime.datetime.strptime(data.loc[6]["Date"],"%a, %b %d, %Y"))).days

0.1


In [2]:
3+4

7